# Coursera Capstone
### Segmenting and Clustering Neighborhoods in Toronto PART2

In [1]:
import numpy as np 
import pandas as pd 
from bs4 import BeautifulSoup
import requests

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

In [3]:
class Scrapy:
       
        def parse_url(self, url):
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'lxml')
            return [(self.parse_html_table(table))\
                    for table in soup.find_all('table', class_="wikitable sortable")]  
    
        def parse_html_table(self, table):
            n_columns = 0
            n_rows=0
            column_names = []
            for row in table.find_all('tr'):
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        n_columns = len(td_tags)
                        
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text())
    
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
    
            columns = column_names if len(column_names) > 0 else range(0,n_columns)
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text()
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1
                    
            for col in df:
                try:
                    df[col] = df[col].astype(float)
                except ValueError:
                    pass
            
            return df

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
hp = Scrapy()
table = hp.parse_url(url)[0] 
table.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Not assigned\n
9,M8A,Not assigned,Not assigned\n


#### Remove the "Not assigned"

In [5]:
table = table[table.Borough != 'Not assigned']
table.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Not assigned\n
10,M9A,Etobicoke,Islington Avenue\n
11,M1B,Scarborough,Rouge\n
12,M1B,Scarborough,Malvern\n


### Cleanup

In [6]:
print(list(table))
table = table.replace('\n',' ', regex=True)
table.head(10)

['Postcode', 'Borough', 'Neighbourhood\n']


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [7]:
table = table[table['Neighbourhood\n'] != 'Not assigned']
table.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


### Groupby Nieghborhood Postcodes

In [8]:
df = table.groupby(['Postcode','Borough'])['Neighbourhood\n'].apply(lambda x: ", ".join(x.astype(str))).reset_index()
df = df.sample(frac=1).reset_index(drop=True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M4E,East Toronto,The Beaches
1,M9M,North York,"Emery , Humberlea"
2,M6E,York,Caledonia-Fairbanks
3,M1W,Scarborough,"L'Amoreaux West , Steeles West"
4,M3N,North York,Downsview Northwest
5,M9A,Etobicoke,Islington Avenue
6,M1G,Scarborough,Woburn
7,M5P,Central Toronto,"Forest Hill North , Forest Hill West"
8,M4R,Central Toronto,North Toronto West
9,M5C,Downtown Toronto,St. James Town


In [9]:
print(df.shape)


(103, 3)


## END Part 1

### Geospacial Data

In [10]:
url2="http://cocl.us/Geospatial_data"
geo_data=pd.read_csv(url2)
geo_data.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [11]:
print(list(df))
print(list(geo_data))

full_table = df.set_index('Postcode').join(geo_data.set_index('Postal Code'))
full_table = full_table.sample(frac=1).reset_index(drop=True)
full_table.head(20)

['Postcode', 'Borough', 'Neighbourhood\n']
['Postal Code', 'Latitude', 'Longitude']


,Borough,Neighbourhood,Latitude,Longitude
0,Etobicoke,"Humber Bay , King's Mill Park , Kingsway Park ...",43.636258,-79.498509
1,Etobicoke,"Humber Bay Shores , Mimico South , New Toronto",43.605647,-79.501321
2,West Toronto,"Runnymede , Swansea",43.651571,-79.484450
3,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188
4,North York,"Bedford Park , Lawrence Manor East",43.733283,-79.419750
5,East Toronto,"The Beaches West , India Bazaar",43.668999,-79.315572
6,North York,Bayview Village,43.786947,-79.385975
7,Central Toronto,North Toronto West,43.715383,-79.405678
8,Downtown Toronto,"First Canadian Place , Underground city",43.648429,-79.382280
9,Central Toronto,Roselawn,43.711695,-79.416936


## END Part 2